In [15]:
!pip install catboost
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, r2_score
from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
df = pd.read_csv("/content/drive/MyDrive/ML_kurs2022/data/train_dataset_train.csv",nrows = 100000)
df

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,7CAE7AE97A4A86C6CAD7658BECD417A3,Социальная карта москвича,386,Нагорная,9010,Нагорная,9,Серпуховско-Тимирязевская,2022-09-12 09:01:31,670.483333,1015
99996,99997,A3E49F5C9B42FBA60C5333CBFF3CE04E,СК учащегося 30 дней,300,Рязанский проспект (Восточный),7016,Рязанский проспект,7,Таганско-Краснопресненская,2022-09-12 09:01:31,537.316667,7015
99997,99998,6A316D059D8E67C83165DAD07D6D4B18,30 дней,109,Войковская ( Северный ),2006,Войковская,2,Замоскворецкая,2022-09-12 09:01:31,707.716667,6008
99998,99999,116262B31A4C71884EAC142CEA331E3B,90 дней ЕДИНЫЙ ТК,276,Новые Черёмушки (Южный),6013,Новые Черёмушки,6,Калужско-Рижская,2022-09-12 09:01:31,566.066667,8008


In [17]:
df["station_nm"].unique()

array(['Лефортово', 'Войковская', 'Аэропорт', 'Косино', 'Котельники',
       'Сокол', 'Новогиреево', 'Кунцевская', 'Проспект Вернадского',
       'Динамо', 'Черкизовская', 'Отрадное', 'Крылатское', 'Мнёвники',
       'Хорошёвская', 'Селигерская', 'Павелецкая', 'Первомайская',
       'Орехово', 'Тургеневская', 'Выставочная', 'Кропоткинская',
       'Площадь Ильича', 'Мякинино', 'Лухмановская', 'Свиблово',
       'Бутырская', 'Петровско-Разумовская', 'Владыкино', 'Комсомольская',
       'Таганская', 'Проспект Мира', 'Шипиловская', 'Третьяковская',
       'Выхино', 'Каширская', 'Савёловская', 'Семёновская',
       'Рязанский проспект', 'Чертановская', 'Печатники', 'Балтийская',
       'Преображенская площадь', 'Севастопольская', 'Тимирязевская',
       'Перово', 'Панфиловская', 'Борисово', 'ВДНХ', 'Новые Черёмушки',
       'Партизанская', 'Ботанический сад', 'Лубянка', 'Спартак',
       'Солнцево', 'Аннино', 'Нижегородская', 'Бульвар Рокоссовского',
       'Смоленская', 'Новокосино', 'Лок

In [18]:
date = df["pass_dttm"]

def date2num(str, isDay):
  days = []
  hours = []
  for i in str:
    day = i.split(" ")[0]
    hour = i.split(" ")[1]
    day = day.split("-")[2]
    hour = hour.split(":")[0]
    days.append(int(day))
    hours.append(int(hour))
  if isDay:
    return days
  else:
    return hours
    
df["days"] = date2num(date, 1)
df["hours"] = date2num(date, 0)
df = df.drop(["pass_dttm","id","entrance_id","ticket_id","station_id","line_id"],axis = 1)
df

,ticket_type_nm,entrance_nm,station_nm,line_nm,time_to_under,label,days,hours
0,Пропуск FacePay,Лефортово БКЛ,Лефортово,Большая кольцевая,216.316667,8001,12,5
1,СК учащегося 30 дней,Войковская ( Южный ),Войковская,Замоскворецкая,648.183333,9011,12,5
2,БСК дружинника г.Москвы,Войковская ( Южный ),Войковская,Замоскворецкая,865.333333,7022,12,5
3,30 дней,Войковская ( Южный ),Войковская,Замоскворецкая,1048.233333,2022,12,5
4,КОШЕЛЕК,Войковская ( Южный ),Войковская,Замоскворецкая,965.600000,2017,12,5
...,...,...,...,...,...,...,...,...
99995,Социальная карта москвича,Нагорная,Нагорная,Серпуховско-Тимирязевская,670.483333,1015,12,9
99996,СК учащегося 30 дней,Рязанский проспект (Восточный),Рязанский проспект,Таганско-Краснопресненская,537.316667,7015,12,9
99997,30 дней,Войковская ( Северный ),Войковская,Замоскворецкая,707.716667,6008,12,9
99998,90 дней ЕДИНЫЙ ТК,Новые Черёмушки (Южный),Новые Черёмушки,Калужско-Рижская,566.066667,8008,12,9


In [19]:
from sklearn.feature_extraction import FeatureHasher

In [20]:
ticket = FeatureHasher(n_features=5, input_type='string')
hf = ticket.fit_transform(df['ticket_type_nm']).toarray()
df = pd.concat([df, pd.DataFrame(hf)], axis=1)

station = FeatureHasher(n_features=5, input_type='string')
hf = station.fit_transform(df['station_nm']).toarray()
df = pd.concat([df, pd.DataFrame(hf)], axis=1)

line = FeatureHasher(n_features=10, input_type='string')
hf = line.fit_transform(df['line_nm']).toarray()
df = pd.concat([df, pd.DataFrame(hf)], axis=1)

entrance = FeatureHasher(n_features=10, input_type='string')
hf = line.fit_transform(df['entrance_nm']).toarray()
df = pd.concat([df, pd.DataFrame(hf)], axis=1)

df = df.drop(["ticket_type_nm","station_nm","line_nm","entrance_nm"],axis=1)
df

,time_to_under,label,days,hours,0,1,2,3,4,0,...,0,1,2,3,4,5,6,7,8,9
0,216.316667,8001,12,5,1.0,-2.0,-1.0,-3.0,2.0,2.0,...,3.0,-2.0,1.0,0.0,-1.0,-2.0,0.0,0.0,1.0,1.0
1,648.183333,9011,12,5,0.0,-2.0,2.0,3.0,1.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
2,865.333333,7022,12,5,-1.0,-5.0,2.0,1.0,0.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
3,1048.233333,2022,12,5,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
4,965.600000,2017,12,5,-2.0,-1.0,0.0,2.0,2.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,670.483333,1015,12,9,1.0,-3.0,1.0,-1.0,-3.0,1.0,...,1.0,0.0,0.0,0.0,-2.0,0.0,0.0,-1.0,2.0,0.0
99996,537.316667,7015,12,9,0.0,-2.0,2.0,3.0,1.0,-4.0,...,-1.0,-1.0,1.0,-2.0,-1.0,-5.0,0.0,-2.0,2.0,1.0
99997,707.716667,6008,12,9,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,0.0,0.0,3.0,2.0,-1.0,-3.0,-1.0,-2.0,1.0,0.0
99998,566.066667,8008,12,9,-1.0,0.0,1.0,4.0,-1.0,0.0,...,-1.0,0.0,2.0,-1.0,-1.0,-1.0,1.0,0.0,3.0,1.0


In [21]:
columns = ["time_to_under","label","days","hours"]
data = df.values

for i in range(len(df.columns)):
  if(type(df.columns[i]) == type(5)):
    columns.append("x" + str(i-3))
df = pd.DataFrame(data, columns = columns)
df

,time_to_under,label,days,hours,x1,x2,x3,x4,x5,x6,...,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30
0,216.316667,8001.0,12.0,5.0,1.0,-2.0,-1.0,-3.0,2.0,2.0,...,3.0,-2.0,1.0,0.0,-1.0,-2.0,0.0,0.0,1.0,1.0
1,648.183333,9011.0,12.0,5.0,0.0,-2.0,2.0,3.0,1.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
2,865.333333,7022.0,12.0,5.0,-1.0,-5.0,2.0,1.0,0.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
3,1048.233333,2022.0,12.0,5.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
4,965.600000,2017.0,12.0,5.0,-2.0,-1.0,0.0,2.0,2.0,-1.0,...,0.0,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,670.483333,1015.0,12.0,9.0,1.0,-3.0,1.0,-1.0,-3.0,1.0,...,1.0,0.0,0.0,0.0,-2.0,0.0,0.0,-1.0,2.0,0.0
99996,537.316667,7015.0,12.0,9.0,0.0,-2.0,2.0,3.0,1.0,-4.0,...,-1.0,-1.0,1.0,-2.0,-1.0,-5.0,0.0,-2.0,2.0,1.0
99997,707.716667,6008.0,12.0,9.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,0.0,0.0,3.0,2.0,-1.0,-3.0,-1.0,-2.0,1.0,0.0
99998,566.066667,8008.0,12.0,9.0,-1.0,0.0,1.0,4.0,-1.0,0.0,...,-1.0,0.0,2.0,-1.0,-1.0,-1.0,1.0,0.0,3.0,1.0


In [22]:
df2 = df.drop(["time_to_under","label"],axis=1)
kmeans = KMeans(n_clusters=3, random_state=42).fit(df2)
cluster = kmeans.predict(df2)
df2['cluster'] = cluster

In [23]:
df2

,days,hours,x1,x2,x3,x4,x5,x6,x7,x8,...,x22,x23,x24,x25,x26,x27,x28,x29,x30,cluster
0,12.0,5.0,1.0,-2.0,-1.0,-3.0,2.0,2.0,-2.0,0.0,...,-2.0,1.0,0.0,-1.0,-2.0,0.0,0.0,1.0,1.0,2
1,12.0,5.0,0.0,-2.0,2.0,3.0,1.0,-1.0,0.0,-1.0,...,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0,0
2,12.0,5.0,-1.0,-5.0,2.0,1.0,0.0,-1.0,0.0,-1.0,...,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0,0
3,12.0,5.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,0.0,-1.0,...,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0,0
4,12.0,5.0,-2.0,-1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,...,1.0,3.0,0.0,-1.0,-3.0,0.0,-2.0,-1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,12.0,9.0,1.0,-3.0,1.0,-1.0,-3.0,1.0,0.0,-1.0,...,0.0,0.0,0.0,-2.0,0.0,0.0,-1.0,2.0,0.0,2
99996,12.0,9.0,0.0,-2.0,2.0,3.0,1.0,-4.0,-2.0,-1.0,...,-1.0,1.0,-2.0,-1.0,-5.0,0.0,-2.0,2.0,1.0,1
99997,12.0,9.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,0.0,-1.0,...,0.0,3.0,2.0,-1.0,-3.0,-1.0,-2.0,1.0,0.0,0
99998,12.0,9.0,-1.0,0.0,1.0,4.0,-1.0,0.0,0.0,2.0,...,0.0,2.0,-1.0,-1.0,-1.0,1.0,0.0,3.0,1.0,2


In [24]:
xl = df['label']
yt = df["time_to_under"]
clf = CatBoostClassifier(iterations=25)
clf.fit(df2, xl)

Learning rate set to 0.5
0:	learn: 5.3154481	total: 13s	remaining: 5m 12s
1:	learn: 5.1369907	total: 28.3s	remaining: 5m 25s
2:	learn: 5.0615705	total: 42.9s	remaining: 5m 14s
3:	learn: 4.9980568	total: 58.2s	remaining: 5m 5s
4:	learn: 4.9456173	total: 1m 14s	remaining: 4m 56s
5:	learn: 4.9180701	total: 1m 28s	remaining: 4m 41s
6:	learn: 4.8750255	total: 1m 43s	remaining: 4m 26s
7:	learn: 4.8521896	total: 1m 58s	remaining: 4m 11s
8:	learn: 4.8164629	total: 2m 18s	remaining: 4m 5s
9:	learn: 4.7965170	total: 2m 32s	remaining: 3m 49s
10:	learn: 4.7695269	total: 2m 47s	remaining: 3m 33s
11:	learn: 4.7460744	total: 3m 2s	remaining: 3m 17s
12:	learn: 4.7244668	total: 3m 17s	remaining: 3m 2s
13:	learn: 4.7089390	total: 3m 32s	remaining: 2m 46s
14:	learn: 4.6872504	total: 3m 47s	remaining: 2m 31s
15:	learn: 4.6675911	total: 4m 2s	remaining: 2m 16s
16:	learn: 4.6505507	total: 4m 16s	remaining: 2m
17:	learn: 4.6296948	total: 4m 31s	remaining: 1m 45s
18:	learn: 4.6169392	total: 4m 46s	remaining: 

In [25]:
pred = clf.predict(df2)
reg = CatBoostRegressor(iterations=1000)
reg.fit(df2, yt)

Learning rate set to 0.084758
0:	learn: 157.0287067	total: 24.1ms	remaining: 24.1s
1:	learn: 155.1983582	total: 46.7ms	remaining: 23.3s
2:	learn: 153.6220520	total: 77ms	remaining: 25.6s
3:	learn: 152.2437262	total: 98.8ms	remaining: 24.6s
4:	learn: 151.0848385	total: 119ms	remaining: 23.7s
5:	learn: 150.1496689	total: 140ms	remaining: 23.2s
6:	learn: 149.3213620	total: 163ms	remaining: 23.2s
7:	learn: 148.6243256	total: 190ms	remaining: 23.6s
8:	learn: 148.0722948	total: 214ms	remaining: 23.6s
9:	learn: 147.5796418	total: 236ms	remaining: 23.4s
10:	learn: 147.1545388	total: 259ms	remaining: 23.3s
11:	learn: 146.8082312	total: 283ms	remaining: 23.3s
12:	learn: 146.5241469	total: 302ms	remaining: 22.9s
13:	learn: 146.2712910	total: 323ms	remaining: 22.7s
14:	learn: 146.0726307	total: 344ms	remaining: 22.6s
15:	learn: 145.8303042	total: 366ms	remaining: 22.5s
16:	learn: 145.6573784	total: 393ms	remaining: 22.7s
17:	learn: 145.4786346	total: 425ms	remaining: 23.2s
18:	learn: 145.3634258	t

In [26]:
pred = clf.predict(df2)
pred_reg = reg.predict(df2)

r2 = r2_score(yt, pred_reg)
r2


0.23411547531489885

In [27]:
recall = recall_score(xl, pred, average="micro")
recall

0.08491

In [28]:
score = 0.5*(recall+r2)
score

0.15951273765744942